<a href="https://colab.research.google.com/github/positivedespair/should_i_wear/blob/master/bot_tg_weather.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install pytelegrambotapi
!pip install urllib3

In [0]:
import telebot
from telebot import types
import requests
from bs4 import BeautifulSoup
import re
import random

TG_KEY = 'insert your key here'
USER_AGENT = 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.181 Safari/537.36'
headers = {'User-Agent': USER_AGENT }
database = {}


In [0]:
def get_weather(cond_en=False):
    try:
        data_ru = requests.get("http://api.openweathermap.org/data/2.5/weather?q=Moscow,ru&appid=9c82f8d680922dfbfeeb2602c42d70bc&lang=ru&units=metric")
        data_ru = data_ru.json()

        temperature = data_ru['main']['temp']
        feels = data_ru['main']['feels_like']
        condition_ru = data_ru['weather'][0]['description']    
        if cond_en:
            data_en = requests.get("http://api.openweathermap.org/data/2.5/weather?q=Moscow,ru&appid=9c82f8d680922dfbfeeb2602c42d70bc&lang=en&units=metric")
            data_en = data_en.json()
            condition_en = data_en['weather'][0]['description']
            return temperature, feels, condition_ru, condition_en
        return temperature, feels, condition_ru
    except: 
        raise(ValueError)

In [0]:
def look(feels, condition_en, gender):
    search = 'street style look ' + gender
    if float(feels) < -10:
        search += 'cool winter'
    elif float(feels) < 0 or 'snow' in condition_en:
        search += 'coat'
    elif 'drizzle' or 'rain' in condition_en:
        search += 'rain'
    elif float(feels) < 10:
        search += 'cool spring'
    elif float(feels) < 16:
        search += 'trench'
    elif float(feels) < 20:
        search += 't shirt'
    elif float(feels) < 30:
        search += 'summer'
    else:
        search += 'hot summer'
    query = search.replace(' ', '+')
    url = 'https://www.google.com/search?q='+ query + '&source=lnms&tbm=isch&tbs=sbd:0'
    resp = requests.get(url, headers = headers)
    if resp.status_code == 200:
        soup = BeautifulSoup(resp.content, "html.parser")
        res = [n for n in re.findall('"([a-zA-Z0-9_./:-]+.(?:jpg|jpeg))",', str(soup))]
        res = res[:10]
        i = random.randint(0, 9)
        return res[i]
    else:
        raise(ValueError)


In [0]:
bot = telebot.TeleBot(TG_KEY)

keyboard_gender = telebot.types.ReplyKeyboardMarkup()
keyboard_gender.row('Мужской', 'Женский')
keyboard_act = telebot.types.ReplyKeyboardMarkup()
keyboard_act.row('Покажи погоду', 'Что стоит надеть?')


@bot.message_handler(commands=['start'])
def send_welcome(message):
    if 'ids' not in database:
        chat_id = message.chat.id
        database['ids'] = message.from_user.id
        m = bot.reply_to(message, text='Укажи свой пол', reply_markup=keyboard_gender)
        bot.register_next_step_handler(m, gender)
    else:
        m = bot.reply_to(message, text='Чем я могу помочь?', reply_markup=keyboard_act)
        bot.register_next_step_handler(m, action)

@bot.message_handler(func=lambda message: True)
def before_start(message):
    try:
        bot.send_message(message.from_user.id, 'Привет!\nЯ могу подсказать, какая сейчас погода в Москве и посоветовать, что надеть, чтобы комфортно чуствовать себя на улице.\nДля начала работы нажми /start')
    except:
        bot.reply_to(message, 'Что-то пошло не так, попробуй еще раз')


def gender(message):
    try:
        if message.text == 'Мужской' or message.text == 'Женский':
            if message.text == 'Мужской':
                database['gender'] = 'man'
            elif message.text == 'Женский':
                database['gender'] = 'women'
            m = bot.reply_to(message, text='Чем я могу помочь?', reply_markup=keyboard_act)
            bot.register_next_step_handler(m, action)
        else:
            msg1 = bot.send_message(message.chat.id, 'Неизвестная команда')
            msg2 = bot.reply_to(msg1, "Укажи свой пол", reply_markup=keyboard_gender)
            bot.register_next_step_handler(msg2, gender)
        
    except Exception as e:
        bot.send_message(message.chat.id, "Что-то пошло не так...")

def action(message):
    if message.text == 'Что стоит надеть?':
        bot.send_message(message.from_user.id, 'Секундочку, ищу для тебя самый модный look!')
        _, feel, _, cond_en = get_weather(True)
        try:
            link = look(feel, cond_en, database['gender'])
            msg1 = bot.send_photo(message.from_user.id, link)
            msg2 = bot.reply_to(msg1, text='Что-нибудь еще?', reply_markup=keyboard_act)
            bot.register_next_step_handler(msg2, action)
        except:
            bot.send_message(message.chat.id, "Не удалось подобрать look")
    elif message.text == 'Покажи погоду':
        try:
            temp, feel, cond = get_weather()
            msg1 = bot.send_message(message.from_user.id, 'Температура на улице: ' + str(temp) + '\nОщущается как: ' + str(feel) + '\nОписание: ' + cond)
            msg2 = bot.reply_to(msg1, text='Что-нибудь еще?', reply_markup=keyboard_act)
            bot.register_next_step_handler(msg2, action)
        except:
            bot.send_message(message.chat.id, "Что-то пошло не так...")
    else:
        if message.text == '/start':
            msg1 = bot.reply_to(message, text='Чем я могу помочь?', reply_markup=keyboard_act)
            bot.register_next_step_handler(msg1, action)
        else:
            msg1 = bot.send_message(message.from_user.id, 'Я не знаю такой команды :(')
            msg2 = bot.reply_to(msg1, text='Чем я могу помочь?', reply_markup=keyboard_act)
            bot.register_next_step_handler(msg2, action)

bot.polling()